In [1]:
%pip install huggingface_hub[hf-cli]
%pip install transformers
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\tbhro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\tbhro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\tbhro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# from huggingface_hub import login
# from datasets import load_dataset

# login()

# dataset = load_dataset("BabyLM-community/babylm-zho")

# # dataset["train"].to_csv("full_train_dataset.csv", index=False)

# # create train / eval split from the single train split
# split_ds = dataset["train"].train_test_split(test_size=0.1, seed=42)

# split_ds["train"].to_csv("train_dataset.csv", index=False)
# split_ds["test"].to_csv("eval_dataset.csv", index=False)

# # maybe don't split the test dataset at random, mixes up values 


Creating CSV from Arrow format:   0%|          | 0/184 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/21 [00:00<?, ?ba/s]

52908549

In [3]:
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\tbhro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
from transformers import set_seed
from transformers import AutoConfig, AutoModelForCausalLM, DebertaV2Tokenizer
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
import torch
import os

# Force CPU usage
os.environ["CUDA_VISIBLE_DEVICES"] = ""
torch.cuda.is_available = lambda: False

def preprocess_logits_for_metrics(logits, labels):
    """Extract predicted token IDs from model logits for evaluation"""
    pred_ids = torch.argmax(logits, dim=-1)  # Get the token with highest probability
    return pred_ids


def compute_metrics(eval_pred):
    """Calculate accuracy by comparing predictions with true labels"""
    logits, labels = eval_pred
    predictions = logits.flatten()  # Flatten to 1D array
    labels = labels.flatten()
    
    # Only consider non-padding tokens (labels != -100 are actual tokens)
    mask = labels != -100
    labels = labels[mask]
    predictions = predictions[mask]

    # Calculate accuracy
    correct = labels == predictions
    accuracy = correct.sum() / float(len(correct))
    return {"acc": accuracy}

In [4]:
from transformers import DebertaV2Tokenizer, AutoConfig, AutoModelForCausalLM

# Load your custom BPE tokenizer
tokenizer = DebertaV2Tokenizer("zho_model_data/pinyin_bpe.model")
print("Vocab size:", tokenizer.vocab_size)

# Build GPT-2-style config using your vocab size
model_name = "openai-community/gpt2"
config = AutoConfig.from_pretrained(model_name)

config.hidden_size = 384
config.intermediate_size = 1280
config.vocab_size = tokenizer.vocab_size  # MUST match your tokenizer

model = AutoModelForCausalLM.from_config(config)
model.to("cpu")   # explicitly place model on CPU



Vocab size: 500


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(500, 384)
    (wpe): Embedding(1024, 384)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=1152, nx=384)
          (c_proj): Conv1D(nf=384, nx=384)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=1536, nx=384)
          (c_proj): Conv1D(nf=384, nx=1536)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=384, out_features=500, bias=False)
)

In [5]:
# continuation of above cell

print("Number of params:", sum(p.numel() for p in model.parameters()))

# Load your dataset
from datasets import load_dataset

raw_dataset = load_dataset(
    "csv",
    data_files="sample_text_first_letters.csv",
    split="train",
    column_names=[
        "text", "doc-id", "category", "data-source", "script",
        "age-estimate", "license", "misc", "num-tokens", "language"
    ],
    header=0      # <-- THIS SKIPS THE HEADER ROW
)

raw_dataset = raw_dataset.remove_columns([
    "doc-id", "category", "data-source", "script",
    "age-estimate", "license", "misc", "num-tokens", "language"
])


dataset = raw_dataset.train_test_split(test_size=0.1)

print(dataset)

print()
print(dataset['train'][0])
print(dataset['test'][0])


Number of params: 21879552
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 36527
    })
    test: Dataset({
        features: ['text'],
        num_rows: 4059
    })
})

{'text': 'tzgrnwqygydzmbftscldbcjndqddxwzdwznxmzhsmhdnbmykbqwwzdwdndgqbybnwwmsgfcltdxzxlnxfctgyczgdnmrwmljjdjhdmydrnshylswbnmdybzzyzsfctgyczcwwmysnychdzzrgnszwnkyxznkyxzawygqlhbksnmynmznrgnmzlwhxrjsnfenwkwddxlshwnzsnmdwsznktwzmzzjssnqzmcdnndnndyddmygsdwdxymqswyzdzdmdnmwdscxgdsjslbzdshwdlbzgrbmysjdszjhssfddwhyjhgmdlbysjsqsnyhzylbzswnryblwclmyzygzgnrbbnbbnywnmyryjqnqtrhswdtzylybnatyxbgybrzmknndnzjhjapbsndnrnbhrtyezdsndcsytpdsnewmwmzsgsssssmylkdwmsrkywnndlbkyzynennqsygbgjmdnrwmdzwmdzjkdhjzylzlsywljnwrtnnbzzqywnmyrysynywnadrswjdnjggwdhmwyddxclbxxbrsssywmzsgdxnzmkyasbdnrzmkynnnagwmnagwmsyndysswywwanedbdljyyszhdrgbsnjswdcwxzjmbwzjdgcjdnsgdhmrgwhlnjswnjslkhllkzlxzwhllsdnnbgtlmdfdgydssjcnhxsycmzbnjfgclxjwmhxxzydnstxmhdwstmwyzmgstwsmydgstnwxqjylthjsbbmbbywhcezgnkqxxbhedjdsjmhqfqqdhznjfnjfhjfhzmk

In [6]:
# check tokenization
from transformers import DebertaV2Tokenizer

# load your trained tokenizer
tokenizer = DebertaV2Tokenizer("zho_model_data/pinyin_bpe.model")  # adjust path if needed

# grab one example from the dataset
sample = dataset["train"][0]["text"]
print("RAW TEXT:")
print(sample)

# tokenize with HF tokenizer
tokens = tokenizer.tokenize(sample)
print("\nTOKENS:")
print(tokens)

# show the corresponding IDs
ids = tokenizer.encode(sample, add_special_tokens=False)
print("\nTOKEN IDS:")
print(ids[:50])  # first 50, just to keep output short

# map IDs back to pieces (to see merged BPE units clearly)
pieces = tokenizer.convert_ids_to_tokens(ids[:50])
print("\nPIECES FROM IDS:")
print(pieces)


RAW TEXT:
tzgrnwqygydzmbftscldbcjndqddxwzdwznxmzhsmhdnbmykbqwwzdwdndgqbybnwwmsgfcltdxzxlnxfctgyczgdnmrwmljjdjhdmydrnshylswbnmdybzzyzsfctgyczcwwmysnychdzzrgnszwnkyxznkyxzawygqlhbksnmynmznrgnmzlwhxrjsnfenwkwddxlshwnzsnmdwsznktwzmzzjssnqzmcdnndnndyddmygsdwdxymqswyzdzdmdnmwdscxgdsjslbzdshwdlbzgrbmysjdszjhssfddwhyjhgmdlbysjsqsnyhzylbzswnryblwclmyzygzgnrbbnbbnywnmyryjqnqtrhswdtzylybnatyxbgybrzmknndnzjhjapbsndnrnbhrtyezdsndcsytpdsnewmwmzsgsssssmylkdwmsrkywnndlbkyzynennqsygbgjmdnrwmdzwmdzjkdhjzylzlsywljnwrtnnbzzqywnmyrysynywnadrswjdnjggwdhmwyddxclbxxbrsssywmzsgdxnzmkyasbdnrzmkynnnagwmnagwmsyndysswywwanedbdljyyszhdrgbsnjswdcwxzjmbwzjdgcjdnsgdhmrgwhlnjswnjslkhllkzlxzwhllsdnnbgtlmdfdgydssjcnhxsycmzbnjfgclxjwmhxxzydnstxmhdwstmwyzmgstwsmydgstnwxqjylthjsbbmbbywhcezgnkqxxbhedjdsjmhqfqqdhznjfnjfhjfhzmkdbrdhzwwmfsrlywywygnrnzszgswnaslygnrsmwkznaslsdwtsstfbsbhdwdxbnhythyknlbyzdlwgstgmyrtbkhllgbqnhzdxsmwzdwzdwacrlzxrzlwtqnybsnnladnrdbgkjybnyynhssgnrmnhysmyanylwgfwbpwwzmymfylnwxlhdzbfxyzmnxwxtzhdsjryrblwe

In [9]:
# check for unknown 
from collections import Counter

unk_id = tokenizer.unk_token_id
unk_count = 0
total_tokens = 0

for i in range(len(dataset["train"])):  # or len(dataset["train"]), but start small
    text = dataset["train"][i]["text"]
    ids = tokenizer.encode(text, add_special_tokens=False)
    total_tokens += len(ids)
    unk_count += sum(1 for t in ids if t == unk_id)

print("UNK rate:", unk_count / total_tokens)


UNK rate: 0.0
